## Microsoft-hosted Megadetector API Test

Adapted from http://dolphinvm.westus2.cloudapp.azure.com/ai4e/notebooks/cameratrap-sync-api-demo.html#Camera-trap-detection-API-demo

### Setup
#### 1) Make sure venv is running and requirements.txt are installed (see README.md)
#### 2) Load API key from .env file
Make sure you have a .env file somewhere in the project directory with the API key set to enironmental variable "MEGADETECTOR_API_KEY"

In [ ]:
%load_ext dotenv
%dotenv

import os
MEGADETECTOR_API_KEY = os.getenv('MEGADETECTOR_API_KEY')

In [ ]:
import requests
import json
import matplotlib.pyplot as plt

from io import BytesIO
from PIL import Image,ImageDraw
from azure.storage.blob import ContainerClient,BlobClient

%autosave 0

demo_image_account_name = 'cameratrapblobs'
demo_image_container_name = 'cameratrapblobcontainer'
demo_image_account_url = 'https://' + demo_image_account_name + '.blob.core.windows.net/'
demo_image_blob_root = demo_image_account_url + demo_image_container_name

demo_image_container_client = ContainerClient(account_url=demo_image_account_url, 
                                         container_name=demo_image_container_name,
                                         credential=None)

api_subscription_key = MEGADETECTOR_API_KEY

api_base_url = 'https://aiforearth.azure-api.net/api/v1/camera-trap/sync/'
api_detection_url = api_base_url + '/detect'
api_version_url = api_base_url + '/detector_model_version'

min_confidence_to_retrieve = 0.5
min_confidence_to_display = 0.8

### API health/version check¶

In [ ]:
headers = { 'Ocp-Apim-Subscription-Key': api_subscription_key }
version_info = requests.get(api_version_url,headers=headers)
version_info.text

### Functions

In [ ]:
def get_images():
    
    images_data = []
    generator = demo_image_container_client.list_blobs()

    for blob in generator:                  
        blob_client = BlobClient(demo_image_account_url,demo_image_container_name,blob.name)
        download_stream = blob_client.download_blob()
        images_data.append({'name' : blob.name, 'data': download_stream.readall()})
        # print('Read {} bytes'.format(len(images_data[-1]['data'])))
    
    return images_data   


def draw_bounding_box_on_image(image,ymin,xmin,ymax,xmax):

    color = 'red'
    draw = ImageDraw.Draw(image)
    im_width, im_height = image.size
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                  ymin * im_height, ymax * im_height)
    draw.line([(left, top), (left, bottom), (right, bottom),
               (right, top), (left, top)], width=4, fill=color)
    
    
def draw_raw_images():  
    
    fig = plt.figure(figsize=(20,35))

    num_images = len(images)
    
    columns = 2

    rows = (num_images // 2) + (num_images % 2)
    
    for i in range(len(images)):
        
        image = Image.open(BytesIO(images[i]['data']))        
        axis = plt.subplot(rows,columns, i + 1)     
        axis.imshow(image)
        plt.axis('off')
        plt.axis('tight')
            
    plt.show()

def response_to_json(r):
    
    from requests_toolbelt.multipart import decoder
    res = decoder.MultipartDecoder.from_response(r)
    results = {}
    images = {}

    for part in res.parts:
        
        # 'part' is a BodyPart object with b'Content-Type', and b'Content-Disposition';
        # the latter includes 'name' and 'filename' info

        headers = {}
        for k, v in part.headers.items():
            headers[k.decode(part.encoding)] = v.decode(part.encoding)

        if headers.get('Content-Type', None) == 'image/jpeg':
            c = headers.get('Content-Disposition')
            image_name = c.split('name="')[1].split('"')[0]
            image = Image.open(io.BytesIO(part.content))
            images[image_name] = image
        elif headers.get('Content-Type', None) == 'application/json':
            content_disposition = headers.get('Content-Disposition', '')
            if 'detection_result' in content_disposition:
                results['detection_result'] = json.loads(part.content.decode())
            elif 'classification_result' in content_disposition:
                results['classification_result'] = json.loads(part.content.decode())
                
    # ...for each part
    
    return results,images

def call_api(image_info):
    
    image_bytes = image_info['data']
    image_name = image_info['name']
    
    assert isinstance(image_bytes,bytes)
    file = BytesIO(image_bytes)
    files = {}
    files[image_name] = (image_name, file, 'application/octet-stream')
    
    headers = { 'Ocp-Apim-Subscription-Key':api_subscription_key }
    params = {
        'confidence': min_confidence_to_retrieve,
        'render': False
    }
    
    r = requests.post(api_detection_url, params=params, headers=headers, files=files)
    
    return r

### Retrieve sample images

In [ ]:
images = get_images()
draw_raw_images()

### Select an image

In [ ]:
i_image = 1

### Call API

In [ ]:
r = call_api(images[i_image])
print(r)
results,_ = response_to_json(r)
image_name = images[i_image]['name']
detections = results['detection_result'][image_name]
print('Found {} detection(s)'.format(len(detections)))

# Each detection is:
#
# ymin,xmin,ymax,xmax,confidence,class
# 
# Image coordinates are normalized, with the origin in the upper-left

# print(detections)

### Show results

In [ ]:
from PIL import Image
image = Image.open(BytesIO(images[i_image]['data']))

for detection in detections:
    box = detection[0:4]
    confidence = detection[4]
    clss = detection[5]
    if (confidence >= min_confidence_to_display):
        draw_bounding_box_on_image(image, box[0], box[1], box[2], box[3])
image